Imports

In [4]:
import pandas as pd
from textblob import TextBlob
from tqdm import tqdm
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

Task 1: Sentiment Labeling

In [5]:
#load data
df = pd.read_csv('./data/raw/test.csv')
print(df.isna().mean().sort_values(ascending=False))
df = df.fillna('') 

#combine subject and body into a single text column
df['text'] = df.apply(lambda row: row['body'] if row['body'].strip() else row['Subject'], axis=1)
df = df[df['text'].str.strip() != '']

#label sentiment
def classify_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return 'Positive'
    elif polarity < -0.1:
        return 'Negative'
    else:
        return 'Neutral'

tqdm.pandas()
df['Sentiment'] = df['text'].progress_apply(classify_sentiment)

#save labeled data
os.makedirs('./data/processed', exist_ok=True)
df.to_csv('./data/processed/labeled_messages.csv', index=False)

print(df['Sentiment'].value_counts())

Subject    0.0
body       0.0
date       0.0
from       0.0
dtype: float64


100%|██████████| 2191/2191 [00:01<00:00, 2184.74it/s]

Sentiment
Neutral     1053
Positive     974
Negative     164
Name: count, dtype: int64


Task 2: Exploratory Data Analysis

In [6]:
# Plot style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# Paths
RAW_DIR = Path("./data/raw")
PROC_DIR = Path("./data/processed")
VIZ_DIR = Path("./visualization")
VIZ_DIR.mkdir(exist_ok=True)

#load saved data
df = pd.read_csv(PROC_DIR / "labeled_messages.csv")
print("Rows:", len(df))
df.head()

#date cleaning & parsing
def clean_date(x):
    """Convert '########' or empty strings to NaT, else parse M/D/YYYY."""
    if isinstance(x, str) and x.strip().startswith("#"):
        return pd.NaT
    try:
        return pd.to_datetime(x, format="%m/%d/%Y", errors="coerce")
    except Exception:
        return pd.NaT

df["date_parsed"] = df["date"].apply(clean_date)
#save
df.to_csv(PROC_DIR / "labeled_messages.csv", index=False)

print("Date parsing success rate:", df["date_parsed"].notna().mean())
missing_summary = df.isna().mean().rename("missing_ratio").to_frame()
display(missing_summary)

#sentiment distribution
sns.countplot(x="Sentiment", data=df, order=["Positive", "Neutral", "Negative"])
plt.title("Sentiment Distribution")
plt.tight_layout()
plt.savefig(VIZ_DIR / "sentiment_distribution.png")
plt.close()

#time series sentiment trend
df_time = (
    df.dropna(subset=["date_parsed"])
      .assign(month=lambda d: d["date_parsed"].dt.to_period("M"))
      .groupby(["month", "Sentiment"])
      .size()
      .unstack(fill_value=0)
)

df_time.plot(kind="bar", stacked=False)
plt.title("Monthly Message Count by Sentiment")
plt.xlabel("Month")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(VIZ_DIR / "monthly_sentiment_counts.png")
plt.close()

#top senders by message volume
top_senders = (
    df.groupby("from")["text"].count().sort_values(ascending=False).head(10)
)
sns.barplot(y=top_senders.index, x=top_senders.values, orient="h")
plt.title("Top 10 Employees by # Messages")
plt.xlabel("Message Count")
plt.tight_layout()
plt.savefig(VIZ_DIR / "top_senders.png")
plt.close()

#data summary:
# Shape and column names
print("Shape:", df.shape)
print("\nColumns:\n", df.columns)

# Data types
print("\nData types:\n", df.dtypes)

# Missing value ratio
missing_summary = df.isna().mean().rename("missing_ratio").to_frame()
print("\nMissing ratio per column:\n", missing_summary)

# Distribution of sentiment values
print(df['Sentiment'].value_counts())
print("\nSentiment distribution (normalized):\n", df['Sentiment'].value_counts(normalize=True))

# Drop NA dates and convert to month
df_time = (
    df.dropna(subset=["date_parsed"])
      .assign(month=lambda d: d["date_parsed"].dt.to_period("M"))
      .groupby(["month", "Sentiment"])
      .size()
      .unstack(fill_value=0)
)

print("\nMonthly sentiment trend:\n", df_time.tail(12))  # last 12 months, for brevity

# Who sends the most messages?
top_senders = df['from'].value_counts().head(10)
print("\nTop 10 employees by message count:\n", top_senders)

# Check if any senders only send negative messages
only_negative = df[df["Sentiment"] == "Negative"]["from"].value_counts()
print("\nEmployees with most negative messages:\n", only_negative.head(10))



Rows: 2191
Date parsing success rate: 1.0


,missing_ratio
Subject,0.0
body,0.0
date,0.0
from,0.0
text,0.0
Sentiment,0.0
date_parsed,0.0


Shape: (2191, 7)

Columns:
 Index(['Subject', 'body', 'date', 'from', 'text', 'Sentiment', 'date_parsed'], dtype='object')

Data types:
 Subject                object
body                   object
date                   object
from                   object
text                   object
Sentiment              object
date_parsed    datetime64[ns]
dtype: object

Missing ratio per column:
              missing_ratio
Subject                0.0
body                   0.0
date                   0.0
from                   0.0
text                   0.0
Sentiment              0.0
date_parsed            0.0
Sentiment
Neutral     1053
Positive     974
Negative     164
Name: count, dtype: int64

Sentiment distribution (normalized):
 Sentiment
Neutral     0.480602
Positive    0.444546
Negative    0.074852
Name: proportion, dtype: float64

Monthly sentiment trend:
 Sentiment  Negative  Neutral  Positive
month                                 
2011-01           4       47        40
2011-02           8

Task 3: Employee Score Calculation

In [7]:
# 📍 2. Load Labeled Data
df = pd.read_csv("./data/processed/labeled_messages.csv")
def clean_date(x):
    """Convert '########' or empty strings to NaT, else parse M/D/YYYY."""
    if isinstance(x, str) and x.strip().startswith("#"):
        return pd.NaT
    try:
        return pd.to_datetime(x, format="%m/%d/%Y", errors="coerce")
    except Exception:
        return pd.NaT

df["date_parsed"] = df["date"].apply(clean_date)
#save parsed date column
df.to_csv("./data/processed/labeled_messages.csv", index=False)
df = df.dropna(subset=["date_parsed"])  # Drop rows without date

#map scores
sentiment_map = {"Positive": 1, "Negative": -1, "Neutral": 0}
df["Sentiment_Score"] = df["Sentiment"].map(sentiment_map)

df["YearMonth"] = df["date_parsed"].dt.to_period("M")

#combine monthly scores
monthly_scores = (
    df.groupby(["from", "YearMonth"])["Sentiment_Score"]
    .sum()
    .reset_index()
    .rename(columns={"from": "Employee", "YearMonth": "Month", "Sentiment_Score": "Score"})
)

#save data
print(monthly_scores.head())
monthly_scores.to_csv("./data/processed/monthly_sentiment_scores.csv", index=False)

                      Employee    Month  Score
0  bobette.riner@ipgdirect.com  2010-01      2
1  bobette.riner@ipgdirect.com  2010-02      7
2  bobette.riner@ipgdirect.com  2010-03      4
3  bobette.riner@ipgdirect.com  2010-04      2
4  bobette.riner@ipgdirect.com  2010-05      1


Task 4: Employee ranking

In [8]:
# 📍 2. Load Monthly Scores
df = pd.read_csv("./data/processed/monthly_sentiment_scores.csv")
df["Month"] = pd.PeriodIndex(df["Month"], freq="M")

# ------------------------------------------------------------
# 📍 3. Define Ranking Logic
def get_rankings(group):
    top = (
        group.sort_values(by=["Score", "Employee"], ascending=[False, True])
             .head(3)
             .assign(Rank_Type="Top Positive")
    )
    bottom = (
        group.sort_values(by=["Score", "Employee"], ascending=[True, True])
             .head(3)
             .assign(Rank_Type="Top Negative")
    )
    return pd.concat([top, bottom])

# ------------------------------------------------------------
# 📍 4. Apply Ranking Per Month
rankings = df.groupby("Month", group_keys=False).apply(get_rankings).reset_index(drop=True)

# Preview
print(rankings.head(10))

# ------------------------------------------------------------
# 📍 5. Save Rankings to File
rankings.to_csv("./data/processed/monthly_employee_rankings.csv", index=False)

#get overall top positive and negative employees
rankings = pd.read_csv("./data/processed/monthly_employee_rankings.csv")

# Assign +1 for Top Positive, ‑1 for Top Negative
rankings["point"] = rankings["Rank_Type"].map({"Top Positive": 1, "Top Negative": -1})

# Aggregate points across all months
overall_scores = (
    rankings.groupby("Employee")["point"]
    .sum()
    .reset_index()
    .rename(columns={"point": "Overall_Score"})
)

# Sort for global Top Positive (highest) and Top Negative (lowest)
top_global_positive = (
    overall_scores.sort_values(by=["Overall_Score", "Employee"], ascending=[False, True])
    .head(3)
    .assign(Global_Rank="Top Positive")
)

top_global_negative = (
    overall_scores.sort_values(by=["Overall_Score", "Employee"], ascending=[True, True])
    .head(3)
    .assign(Global_Rank="Top Negative")
)

global_top3 = pd.concat([top_global_positive, top_global_negative])
print(global_top3)

# ------------------------------------------------------------
# 📍 7. Save Global Rankings
global_top3.to_csv("./data/processed/global_top3_employees.csv", index=False)

                      Employee    Month  Score     Rank_Type
0      kayne.coulter@enron.com  2010-01      5  Top Positive
1     patti.thompson@enron.com  2010-01      5  Top Positive
2       don.baughman@enron.com  2010-01      4  Top Positive
3      rhonda.denton@enron.com  2010-01      0  Top Negative
4      johnny.palmer@enron.com  2010-01      1  Top Negative
5  bobette.riner@ipgdirect.com  2010-01      2  Top Negative
6  bobette.riner@ipgdirect.com  2010-02      7  Top Positive
7        john.arnold@enron.com  2010-02      7  Top Positive
8       don.baughman@enron.com  2010-02      6  Top Positive
9      lydia.delgado@enron.com  2010-02      1  Top Negative
                      Employee  Overall_Score   Global_Rank
6      lydia.delgado@enron.com              5  Top Positive
3        john.arnold@enron.com              4  Top Positive
2          eric.bass@enron.com              2  Top Positive
8      rhonda.denton@enron.com             -7  Top Negative
4      johnny.palmer@enron.co

C:\Users\admin\AppData\Local\Temp\ipykernel_9588\2807542822.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rankings = df.groupby("Month", group_keys=False).apply(get_rankings).reset_index(drop=True)


Task 5: Flight Risk

In [9]:
#read data
df = pd.read_csv("./data/processed/labeled_messages.csv")
df["date_parsed"] = pd.to_datetime(df["date_parsed"], errors="coerce")

# Keep only negative messages with a valid date
df_neg = df[(df["Sentiment"] == "Negative") & (df["date_parsed"].notna())]
df_neg = df_neg.sort_values(["from", "date_parsed"])

# ------------------------------------------------------------
# 📍 3. Identify Rolling 30-day Negative Message Clusters
def flag_risk(group):
    risk_dates = []
    dates = group["date_parsed"].tolist()
    for i in range(len(dates)):
        count = 1
        start = dates[i]
        for j in range(i+1, len(dates)):
            if (dates[j] - start).days <= 30:
                count += 1
            else:
                break
        if count >= 4:
            risk_dates.append(start)
    return pd.Series({"At_Risk": len(risk_dates) > 0})

risk_flags = df_neg.groupby("from").apply(flag_risk).reset_index()
risk_flags = risk_flags.rename(columns={"from": "Employee"})

# ------------------------------------------------------------
# 📍 4. Output & Save
print(risk_flags[risk_flags["At_Risk"] == True])
risk_flags.to_csv("./data/processed/flight_risk_employees.csv", index=False)


                      Employee  At_Risk
0  bobette.riner@ipgdirect.com     True
3        john.arnold@enron.com     True
4      johnny.palmer@enron.com     True
6      lydia.delgado@enron.com     True
7     patti.thompson@enron.com     True
8      rhonda.denton@enron.com     True
9         sally.beck@enron.com     True


C:\Users\admin\AppData\Local\Temp\ipykernel_9588\3268918470.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  risk_flags = df_neg.groupby("from").apply(flag_risk).reset_index()


Task 6: Linear Regression Model

In [10]:
#imports
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# load and preprocess data
df = pd.read_csv("./data/processed/labeled_messages.csv")
df["date_parsed"] = pd.to_datetime(df["date_parsed"], errors="coerce")
df = df.dropna(subset=["date_parsed"])

# Sentiment to numeric
sentiment_map = {"Positive": 1, "Negative": -1, "Neutral": 0}
df["Sentiment_Score"] = df["Sentiment"].map(sentiment_map)

# Message features
df["char_count"] = df["text"].astype(str).apply(len)
df["word_count"] = df["text"].astype(str).apply(lambda x: len(x.split()))
df["Month"] = df["date_parsed"].dt.to_period("M")

# Group & Feature Engineering
monthly_df = df.groupby(["from", "Month"]).agg({
    "text": "count",
    "char_count": "mean",
    "word_count": "mean",
    "Sentiment_Score": "sum"
}).reset_index()

monthly_df = monthly_df.rename(columns={
    "from": "Employee",
    "text": "msg_count",
    "char_count": "avg_msg_length",
    "word_count": "avg_word_count",
    "Sentiment_Score": "sentiment_score"
})

# Train/Test Split
features = ["msg_count", "avg_msg_length", "avg_word_count"]
X = monthly_df[features]
y = monthly_df["sentiment_score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# ------------------------------------------------------------
# Evaluation
print("R^2 Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Coefficients
coef_df = pd.DataFrame({
    "Feature": features,
    "Coefficient": model.coef_
})
print(coef_df)

R^2 Score: 0.42765020804350784
RMSE: 1.994649338696962
          Feature  Coefficient
0       msg_count     0.347254
1  avg_msg_length    -0.004865
2  avg_word_count     0.039418
